#Estrutura de Dados - Aula 10

## Fila de Prioridade

Classe PriorityQueueBase: Uma classe base abstrata para uma fila de prioridade.

Classe Item: Uma classe leve para armazenar itens da fila de prioridade como pares chave-valor.

\_\_slots\_\_: Define _key e _value como os únicos atributos permitidos para instâncias da classe Item, economizando memória.

\_\_init\_\_(self, k, v): Inicializa um item com uma chave k e um valor v.

__lt__(self, other): Define a comparação entre itens baseada em suas chaves.

Método is\_empty(self): Verifica se a fila de prioridade está vazia.

#### Implementando uma fila de prioridade desordenada

####Obs:. Foi utilizado a lista posicional da aula passada (com algumas alterações).

In [ ]:
class PriorityQueueBase:
    """Classe base abstrata para uma fila de prioridade."""

    class Item:
        """Composto leve para armazenar itens da fila de prioridade."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key  # compara itens com base em suas chaves

    def is_empty(self):
        """Retorna True se a fila de prioridade estiver vazia."""
        return len(self) == 0

class DoublyLinkedBase:
    """Uma classe base que fornece uma representação de lista duplamente ligada."""

    class Node:
        """Classe leve e não pública para armazenar um nó duplamente ligado."""
        __slots__ = '_element', '_prev', '_next'

        def __init__(self, element, prev, next):
            self._element = element
            self._prev = prev
            self._next = next

    def __init__(self):
        """Cria uma lista vazia."""
        self._header = self.Node(None, None, None)
        self._trailer = self.Node(None, None, None)
        self._header._next = self._trailer  # trailer está após o header
        self._trailer._prev = self._header  # header está antes do trailer
        self._size = 0  # número de elementos

    def __len__(self):
        """Retorna o número de elementos na lista."""
        return self._size

    def is_empty(self):
        """Retorna True se a lista estiver vazia."""
        return self._size == 0

    def _insert_between(self, e, predecessor, successor):
        """Adiciona o elemento e entre dois nós existentes e retorna o novo nó."""
        newest = self.Node(e, predecessor, successor)  # ligado aos vizinhos
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest

    def _delete_node(self, node):
        """Remove um nó não-sentinela da lista e retorna seu elemento."""
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        element = node._element  # registra o elemento removido
        node._prev = node._next = node._element = None  # obsoleta o nó
        return element  # retorna o elemento removido

class PositionalList(DoublyLinkedBase):
    """Um contêiner sequencial de elementos permitindo acesso posicional."""

    class Position:
        """Uma abstração representando a localização de um único elemento."""
        def __init__(self, container, node):
            """O construtor não deve ser invocado pelo usuário."""
            self._container = container
            self._node = node

        def element(self):
            """Retorna o elemento armazenado nesta Posição."""
            return self._node._element

        def __eq__(self, other):
            """Retorna True se other for uma Posição representando a mesma localização."""
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            """Retorna True se other não representar a mesma localização."""
            return not (self == other)  # oposto de __eq__

    def _validate(self, p):
        """Retorna o nó da posição, ou lança um erro apropriado se inválido."""
        if not isinstance(p, self.Position):
            raise TypeError("p deve ser do tipo Position correto")
        if p._container is not self:
            raise ValueError("p não pertence a este contêiner")
        if p._node._next is None:  # convenção para nós obsoletos
            raise ValueError("p não é mais válido")
        return p._node

    def _make_position(self, node):
        """Retorna a instância Position para o nó fornecido (ou None se sentinela)."""
        if node is self._header or node is self._trailer:
            return None  # não use para sentinelas
        else:
            return self.Position(self, node)

    def first(self):
        """Retorna a primeira Posição na lista (ou None se a lista estiver vazia)."""
        return self._make_position(self._header._next)

    def last(self):
        """Retorna a última Posição na lista (ou None se a lista estiver vazia)."""
        return self._make_position(self._trailer._prev)

    def before(self, p):
        """Retorna a Posição antes de p (ou None se p estiver na frente)."""
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        """Retorna a Posição depois de p (ou None se p estiver no final)."""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Gera uma iteração dos elementos na lista."""
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    def _insert_between(self, e, predecessor, successor):
        """Adiciona o elemento e entre dois nós existentes e retorna a nova Posição."""
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)

    def add_first(self, e):
        """Insere o elemento e no início da lista e retorna a Posição."""
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        """Insere o elemento e no final da lista e retorna a Posição."""
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        """Insere o elemento e antes da Posição p e retorna a nova Posição."""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p, e):
        """Insere o elemento e após a Posição p e retorna a nova Posição."""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)

    def delete(self, p):
        """Remove e retorna o elemento na Posição p."""
        original = self._validate(p)
        return self._delete_node(original)

    def replace(self, p, e):
        """Substitui o elemento na Posição p pelo e e retorna o elemento antigo."""
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value

class UnsortedPriorityQueue(PriorityQueueBase):
    """Uma fila de prioridade mínima implementada com uma lista não ordenada."""

    def _find_min(self):
        """Retorna a posição do item com a chave mínima."""
        if self.is_empty():
            raise ValueError("Priority queue is empty")
        small = self._data.first()
        walk = self._data.after(small)
        while walk is not None:
            if walk.element() < small.element():
                small = walk
            walk = self._data.after(walk)
        return small

    def __init__(self):
        """Cria uma nova fila de prioridade vazia."""
        self._data = PositionalList()

    def __len__(self):
        """Retorna o número de itens na fila de prioridade."""
        return len(self._data)

    def add(self, key, value):
        """Adiciona um par chave-valor."""
        self._data.add_last(self.Item(key, value))

    def min(self):
        """Retorna mas não remove a tupla (k,v) com a chave mínima."""
        p = self._find_min()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove e retorna a tupla (k,v) com a chave mínima."""
        p = self._find_min()
        item = self._data.delete(p)
        return (item._key, item._value)

In [ ]:
# Criando a fila de prioridade
pq = UnsortedPriorityQueue()

In [ ]:
# Adicionando elementos
pq.add(5, 'A')
pq.add(9, 'C')
pq.add(3, 'B')
pq.add(7, 'D')


In [ ]:
# Verificando se a fila está vazia
print("Fila vazia?", pq.is_empty())  # Deve retornar False



Fila vazia? False


In [ ]:
# Obter o número de elementos na fila de prioridade
print("Número de elementos:", len(pq))  # Deve retornar 4


Número de elementos: 4


In [ ]:
# Imprimindo o elemento com a menor chave
print("Min:", pq.min())  # Deve ser (3, 'B')



Min: (3, 'B')


In [ ]:
# Removendo e imprimindo o elemento com a menor chave
print("Remove Min:", pq.remove_min())  # Deve ser (3, 'B')
print("Min:", pq.min())  # Deve ser (5, 'A')



Remove Min: (3, 'B')
Min: (5, 'A')


In [ ]:
# Verificar a prioridade de todos os elementos na fila
print("Elementos na fila:")
for item in pq._data:
    print(f"Chave: {item._key}, Valor: {item._value}")

Elementos na fila:
Chave: 5, Valor: A
Chave: 9, Valor: C
Chave: 7, Valor: D


# Implementando uma fila de prioridade ordenada

In [ ]:
class PriorityQueueBase:
    """Classe base abstrata para uma fila de prioridade."""

    class Item:
        """Composto leve para armazenar itens da fila de prioridade."""
        __slots__ = '_key', '_value'

        def __init__(self, k, v):
            self._key = k
            self._value = v

        def __lt__(self, other):
            return self._key < other._key  # compara itens com base em suas chaves

    def is_empty(self):
        """Retorna True se a fila de prioridade estiver vazia."""
        return len(self) == 0

class DoublyLinkedBase:
    """Uma classe base que fornece uma representação de lista duplamente ligada."""

    class Node:
        """Classe leve e não pública para armazenar um nó duplamente ligado."""
        __slots__ = '_element', '_prev', '_next'

        def __init__(self, element, prev, next):
            self._element = element
            self._prev = prev
            self._next = next

    def __init__(self):
        """Cria uma lista vazia."""
        self._header = self.Node(None, None, None)
        self._trailer = self.Node(None, None, None)
        self._header._next = self._trailer  # trailer está após o header
        self._trailer._prev = self._header  # header está antes do trailer
        self._size = 0  # número de elementos

    def __len__(self):
        """Retorna o número de elementos na lista."""
        return self._size

    def is_empty(self):
        """Retorna True se a lista estiver vazia."""
        return self._size == 0

    def _insert_between(self, e, predecessor, successor):
        """Adiciona o elemento e entre dois nós existentes e retorna o novo nó."""
        newest = self.Node(e, predecessor, successor)  # ligado aos vizinhos
        predecessor._next = newest
        successor._prev = newest
        self._size += 1
        return newest

    def _delete_node(self, node):
        """Remove um nó não-sentinela da lista e retorna seu elemento."""
        predecessor = node._prev
        successor = node._next
        predecessor._next = successor
        successor._prev = predecessor
        self._size -= 1
        element = node._element  # registra o elemento removido
        node._prev = node._next = node._element = None  # obsoleta o nó
        return element  # retorna o elemento removido

class PositionalList(DoublyLinkedBase):
    """Um contêiner sequencial de elementos permitindo acesso posicional."""

    class Position:
        """Uma abstração representando a localização de um único elemento."""
        def __init__(self, container, node):
            """O construtor não deve ser invocado pelo usuário."""
            self._container = container
            self._node = node

        def element(self):
            """Retorna o elemento armazenado nesta Posição."""
            return self._node._element

        def __eq__(self, other):
            """Retorna True se other for uma Posição representando a mesma localização."""
            return type(other) is type(self) and other._node is self._node

        def __ne__(self, other):
            """Retorna True se other não representar a mesma localização."""
            return not (self == other)  # oposto de __eq__

    def _validate(self, p):
        """Retorna o nó da posição, ou lança um erro apropriado se inválido."""
        if not isinstance(p, self.Position):
            raise TypeError("p deve ser do tipo Position correto")
        if p._container is not self:
            raise ValueError("p não pertence a este contêiner")
        if p._node._next is None:  # convenção para nós obsoletos
            raise ValueError("p não é mais válido")
        return p._node

    def _make_position(self, node):
        """Retorna a instância Position para o nó fornecido (ou None se sentinela)."""
        if node is self._header or node is self._trailer:
            return None  # não use para sentinelas
        else:
            return self.Position(self, node)

    def first(self):
        """Retorna a primeira Posição na lista (ou None se a lista estiver vazia)."""
        return self._make_position(self._header._next)

    def last(self):
        """Retorna a última Posição na lista (ou None se a lista estiver vazia)."""
        return self._make_position(self._trailer._prev)

    def before(self, p):
        """Retorna a Posição antes de p (ou None se p estiver na frente)."""
        node = self._validate(p)
        return self._make_position(node._prev)

    def after(self, p):
        """Retorna a Posição depois de p (ou None se p estiver no final)."""
        node = self._validate(p)
        return self._make_position(node._next)

    def __iter__(self):
        """Gera uma iteração dos elementos na lista."""
        cursor = self.first()
        while cursor is not None:
            yield cursor.element()
            cursor = self.after(cursor)

    def _insert_between(self, e, predecessor, successor):
        """Adiciona o elemento e entre dois nós existentes e retorna a nova Posição."""
        node = super()._insert_between(e, predecessor, successor)
        return self._make_position(node)

    def add_first(self, e):
        """Insere o elemento e no início da lista e retorna a Posição."""
        return self._insert_between(e, self._header, self._header._next)

    def add_last(self, e):
        """Insere o elemento e no final da lista e retorna a Posição."""
        return self._insert_between(e, self._trailer._prev, self._trailer)

    def add_before(self, p, e):
        """Insere o elemento e antes da Posição p e retorna a nova Posição."""
        original = self._validate(p)
        return self._insert_between(e, original._prev, original)

    def add_after(self, p, e):
        """Insere o elemento e após a Posição p e retorna a nova Posição."""
        original = self._validate(p)
        return self._insert_between(e, original, original._next)

    def delete(self, p):
        """Remove e retorna o elemento na Posição p."""
        original = self._validate(p)
        return self._delete_node(original)

    def replace(self, p, e):
        """Substitui o elemento na Posição p pelo e e retorna o elemento antigo."""
        original = self._validate(p)
        old_value = original._element
        original._element = e
        return old_value

class SortedPriorityQueue(PriorityQueueBase):
    """Uma fila de prioridade mínima implementada com uma lista ordenada."""

    def __init__(self):
        """Cria uma nova fila de prioridade vazia."""
        self._data = PositionalList()

    def __len__(self):
        """Retorna o número de itens na fila de prioridade."""
        return len(self._data)

    def add(self, key, value):
        """Adiciona um par chave-valor."""
        new_item = self.Item(key, value)
        walk = self._data.last()
        while walk is not None and new_item < walk.element():
            walk = self._data.before(walk)
        if walk is None:
            self._data.add_first(new_item)
        else:
            self._data.add_after(walk, new_item)

    def min(self):
        """Retorna mas não remove a tupla (k,v) com a chave mínima."""
        if self.is_empty():
            raise ValueError("Priority queue is empty")
        p = self._data.first()
        item = p.element()
        return (item._key, item._value)

    def remove_min(self):
        """Remove e retorna a tupla (k,v) com a chave mínima."""
        if self.is_empty():
            raise ValueError("Priority queue is empty")
        item = self._data.delete(self._data.first())
        return (item._key, item._value)

In [ ]:
pq = SortedPriorityQueue()



In [ ]:
# Adicionando elementos
pq.add(5, 'A')
pq.add(9, 'C')
pq.add(3, 'B')
pq.add(7, 'D')



In [ ]:
# Verificando se a fila está vazia
print("Fila vazia?", pq.is_empty())  # Deve retornar False



Fila vazia? False


In [ ]:
# Obter o número de elementos na fila de prioridade
print("Número de elementos:", len(pq))  # Deve retornar 4



Número de elementos: 4


In [ ]:
# Imprimindo o elemento com a menor chave
print("Min:", pq.min())  # Deve ser (3, 'B')



Min: (3, 'B')


In [ ]:
# Removendo e imprimindo o elemento com a menor chave
print("Remove Min:", pq.remove_min())  # Deve ser (3, 'B')
print("Min:", pq.min())  # Deve ser (5, 'A')

Remove Min: (3, 'B')
Min: (5, 'A')


##Lista de Prioridade Não Ordenada em Java


```java
import java.util.ArrayList;
import java.util.List;

public class ListaPrioridadeNaoOrdenada<T> {
    private static class Entrada<T> {
        int prioridade;
        T valor;

        Entrada(int prioridade, T valor) {
            this.prioridade = prioridade;
            this.valor = valor;
        }
    }

    private List<Entrada<T>> lista;

    public ListaPrioridadeNaoOrdenada() {
        lista = new ArrayList<>();
    }

    // Inserção rápida: apenas adiciona no final da lista
    public void inserir(int prioridade, T valor) {
        lista.add(new Entrada<>(prioridade, valor));
    }

    // Remove e retorna o elemento de maior prioridade (menor valor)
    public T remover() {
        if (lista.isEmpty()) return null;

        int idxMaior = 0;
        for (int i = 1; i < lista.size(); i++) {
            if (lista.get(i).prioridade < lista.get(idxMaior).prioridade) {
                idxMaior = i;
            }
        }

        return lista.remove(idxMaior).valor;
    }

    public boolean estaVazia() {
        return lista.isEmpty();
    }
}


```

##Teste da Fila de Prioridade Não Ordenada

```java
public class TesteNaoOrdenada {
    public static void main(String[] args) {
        ListaPrioridadeNaoOrdenada<String> fila = new ListaPrioridadeNaoOrdenada<>();

        fila.inserir(3, "Email comum");
        fila.inserir(1, "Alerta de segurança");
        fila.inserir(5, "Newsletter");
        fila.inserir(2, "Erro de sistema");

        while (!fila.estaVazia()) {
            System.out.println("Removido: " + fila.remover());
        }
    }
}

// SAIDA ESPERADA
// Removido: Alerta de segurança
// Removido: Erro de sistema
// Removido: Email comum
// Removido: Newsletter

```

##Lista de Prioridade Ordenada em Java

```java
import java.util.LinkedList;

public class ListaPrioridadeOrdenada<T> {
    private static class Entrada<T> {
        int prioridade;
        T valor;

        Entrada(int prioridade, T valor) {
            this.prioridade = prioridade;
            this.valor = valor;
        }
    }

    private LinkedList<Entrada<T>> lista;

    public ListaPrioridadeOrdenada() {
        lista = new LinkedList<>();
    }

    // Inserção mantém a lista ordenada (crescente por prioridade)
    public void inserir(int prioridade, T valor) {
        Entrada<T> nova = new Entrada<>(prioridade, valor);

        int i = 0;
        while (i < lista.size() && lista.get(i).prioridade <= prioridade) {
            i++;
        }
        lista.add(i, nova);
    }

    // Remove e retorna o primeiro elemento (maior prioridade = menor valor)
    public T remover() {
        if (lista.isEmpty()) return null;
        return lista.removeFirst().valor;
    }

    public boolean estaVazia() {
        return lista.isEmpty();
    }
}


```

##Teste da Fila de Prioridade Ordenada

```java
public class TesteOrdenada {
    public static void main(String[] args) {
        ListaPrioridadeOrdenada<String> fila = new ListaPrioridadeOrdenada<>();

        fila.inserir(3, "Email comum");
        fila.inserir(1, "Alerta de segurança");
        fila.inserir(5, "Newsletter");
        fila.inserir(2, "Erro de sistema");

        while (!fila.estaVazia()) {
            System.out.println("Removido: " + fila.remover());
        }
    }
}

// SAIDA ESPERADA
// Removido: Alerta de segurança
// Removido: Erro de sistema
// Removido: Email comum
// Removido: Newsletter


```

